In [61]:
import requests
from bs4 import BeautifulSoup

In [62]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'html')

In [63]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of United States cities by population - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pin

In [64]:
table = soup.select('table.wikitable.sortable')

In [65]:
print(table)

[<table class="sortable wikitable sticky-header static-row-numbers col1left col2center" style="text-align:right">
<tbody><tr>
<th>City
</th>
<th><abbr title="State or district">ST</abbr>
</th>
<th>2022<br/>estimate
</th>
<th>2020<br/>census
</th>
<th>Change
</th>
<th colspan="2">2020 land area
</th>
<th colspan="2">2020 density
</th>
<th>Location
</th></tr>
<tr class="boilerplate metadata plainlinks tfd" id="tfd" style="background-color: transparent; padding: 0; font-size:85%; color:#000000; text-align: center; border-bottom:1px solid #AAAAAA; overflow:hidden;">
<th style="background-position:center;line-height:100%;"> </th>
<th style="background-position:center"> </th>
<th style="background-position:center"> </th>
<th style="background-position:center"> </th>
<th style="background-position:center"> </th>
<th style="background-position:center"> </th>
<th style="background-position:center"> </th>
<th style="background-position:center"> </th>
<th style="background-position:center"> </th>

In [66]:
if table:
    # Find all the 'th' elements within the table
    table_titles = [header.get_text(strip=True) for header in table[0].find_all('th')][:8]
    print(table_titles)
else:
    print('No table found')


['City', 'ST', '2022estimate', '2020census', 'Change', '2020 land area', '2020 density', 'Location']


In [67]:
import pandas as pd

In [68]:
# Change the second column name from ST to State
table_titles[1] = 'State'

# Delete the 3, 4, 5, 6 and 8th column
table_titles = table_titles[:2] + table_titles[6:-1]
print(table_titles)

['City', 'State', '2020 density']


In [69]:
# Create a dataframe from the table
df = pd.DataFrame(columns=table_titles)

df

,City,State,2020 density


In [74]:
# Find all rows in the table body
table_body = table[0].find('tbody')
if table_body:
    rows = table_body.find_all('tr')

    # Create a list to store rows
    rows_data = []

    # Iterate through each row and append data to the list
    for row in rows[1:]:  # Skip the first row if it contains header data
        # Find all cells in the row
        cells = row.find_all(['td'])
        
        # Check if there are enough cells to proceed
        if len(cells) >= 9:
            # Extract relevant information by index.
            # Remove the square brackets and anything inside from city 
            city = cells[0].get_text(strip=True).split('[')[0]
            state = cells[1].get_text(strip=True)
            density_cell = cells[8]  # Assuming '2020 density' is in the 9th column but index 8
    
            # Check if the required density cell is present
            if density_cell:
                # Extract the density content.
                # Remove the '/km2' from the end of the string
                density_content = density_cell.get_text(strip=True).replace('/km2', '')
                
                # Check if <sup>2</sup> is present in the content
                if density_cell.find('sup'):
                    # If <sup>2</sup> is present, it's in kilometers squared
                    density = density_content + 'km²'
                    
                    # Append only the necessary columns to the list
                    rows_data.append([city, state, density])
                    
# Convert the list of rows to a DataFrame
df = pd.DataFrame(rows_data, columns=['City', 'State', '2020 density'])

# Print the updated DataFrame
print(df)


            City State 2020 density
0       New York    NY    11,312km²
1    Los Angeles    CA     3,206km²
2        Chicago    IL     4,657km²
3        Houston    TX     1,390km²
4        Phoenix    AZ     1,199km²
..           ...   ...          ...
328       Albany    NY     1,790km²
329     Hesperia    CA       530km²
330  New Bedford    MA     1,951km²
331    Davenport    IA       615km²
332    Daly City    CA     5,329km²

[333 rows x 3 columns]


In [75]:
df.head()

,City,State,2020 density
0,New York,NY,"11,312km²"
1,Los Angeles,CA,"3,206km²"
2,Chicago,IL,"4,657km²"
3,Houston,TX,"1,390km²"
4,Phoenix,AZ,"1,199km²"


In [76]:
# Check for Missing Values:
# Verify if there are any missing values in the DataFrame.
print(df.isnull().sum())

City            0
State           0
2020 density    0
dtype: int64


In [77]:
# Check for Duplicates:
# Verify if there are any duplicate rows in the DataFrame.
print(df.duplicated().sum())

0


In [78]:
# Check Unique Values:
# Inspect unique values in each column to identify any unexpected or inconsistent entries.
for column in df.columns:
    print(f'Unique values in {column}:\n{df[column].unique()}\n')

Unique values in City:
['New York' 'Los Angeles' 'Chicago' 'Houston' 'Phoenix' 'Philadelphia'
 'San Antonio' 'San Diego' 'Dallas' 'Austin' 'Jacksonville' 'San Jose'
 'Fort Worth' 'Columbus' 'Charlotte' 'Indianapolis' 'San Francisco'
 'Seattle' 'Denver' 'Oklahoma City' 'Nashville' 'El Paso' 'Washington'
 'Las Vegas' 'Boston' 'Portland' 'Louisville' 'Memphis' 'Detroit'
 'Baltimore' 'Milwaukee' 'Albuquerque' 'Tucson' 'Fresno' 'Sacramento'
 'Mesa' 'Kansas City' 'Atlanta' 'Colorado Springs' 'Omaha' 'Raleigh'
 'Virginia Beach' 'Long Beach' 'Miami' 'Oakland' 'Minneapolis' 'Tulsa'
 'Bakersfield' 'Tampa' 'Wichita' 'Arlington' 'Aurora' 'New Orleans'
 'Cleveland' 'Anaheim' 'Honolulu' 'Henderson' 'Stockton' 'Riverside'
 'Lexington' 'Corpus Christi' 'Orlando' 'Irvine' 'Cincinnati' 'Santa Ana'
 'Newark' 'Saint Paul' 'Pittsburgh' 'Greensboro' 'Lincoln' 'Durham'
 'Plano' 'Anchorage' 'Jersey City' 'St. Louis' 'Chandler'
 'North Las Vegas' 'Chula Vista' 'Buffalo' 'Gilbert' 'Reno' 'Madison'
 'Fort Wayne'

In [79]:
# Check Descriptive Statistics:
# Examine basic statistics to identify outliers or anomalies in numeric columns.

print(df.describe())

               City State 2020 density
count           333   333          333
unique          320    46          315
top     Springfield    CA     1,887km²
freq              3    75            3


In [80]:
df

,City,State,2020 density
0,New York,NY,"11,312km²"
1,Los Angeles,CA,"3,206km²"
2,Chicago,IL,"4,657km²"
3,Houston,TX,"1,390km²"
4,Phoenix,AZ,"1,199km²"
...,...,...,...
328,Albany,NY,"1,790km²"
329,Hesperia,CA,530km²
330,New Bedford,MA,"1,951km²"
331,Davenport,IA,615km²


In [82]:
# Save the DataFrame to a XLXS file
df.to_excel('citiesByPop.xlsx', index=False)